In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
df = pd.read_csv("data/traindata.csv",sep = "\t",header=None)

In [ ]:
df.head()

In [ ]:
from nltk.parse.corenlp import CoreNLPDependencyParser
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [ ]:
def get_first_link(target_word,parse,dep):
    for i in range(len(parse)):
        if (target_word in u[i][2][0]):
            dep.append(u[i][0][0])
        elif (target_word in u[i][0][0]):
            dep.append(u[i][2][0])
    return dep

In [ ]:
def get_second_link(target_word,parse,dep):
    for i in range(len(parse)):
        if (target_word in u[i][2][0]) & ("NN" not in u[i][0][1] ) & ("DT" not in u[i][0][1]):
            dep.append(u[i][0][0])
        elif (target_word in u[i][0][0]) & ("NN" not in u[i][2][1]) & (("DT" not in u[i][2][1])):
            dep.append(u[i][2][0])
    return dep

In [ ]:
deps = []
for j in range(len(df)):
    try:
        current_dep = []
        parses = dep_parser.parse(df.iloc[j,4].split())
        context = df.iloc[j,2].split()
    #parses_list = list(dep_parser.parse(df.iloc[5,4].split()))
        u= [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses][0]
    except:
        print(j)
    for word in context: 
        get_first_link(word,u,current_dep)
    direct_links = len(current_dep)
    for i in range(direct_links):
        context = current_dep[i]
        get_second_link(context,u,current_dep)
    deps.append(list(set(current_dep)))


In [ ]:
def filter_sentence(dependencies,sentence):
    sent = sentence.split()
    final_sent = ""
    for i in range(len(sent)):
        if sent[i] in dependencies:
            final_sent+=sent[i]+" "
    return final_sent
#Rajouter tous les mots entre les deux????

In [ ]:
df.iloc[:,4] = df.iloc[:,4].str.replace(r'[^\w\s]+', ' ')

In [ ]:
df["terms"] = deps

In [ ]:
df["new_sent"] = df.apply(lambda x : filter_sentence(x["terms"],x[4]),axis=1)

In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.2)

In [ ]:
def get_labels(sent):
    labels = []
    for i in range(len(sent)):
        if(sent[i]=="positive"):
            labels.append(0)
        elif(sent[i]=="negative"):
            labels.append(1)
        else:
            labels.append(2)
    return torch.tensor(labels).unsqueeze(0)

In [ ]:
from transformers import AutoTokenizer
from transformers import AdamW
import torch



tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
#encoding = tokenizer(list(df_test.iloc[:,-1]), return_tensors='pt', padding="longest", truncation=True)

train_encodings = tokenizer(list(df_train.iloc[:,-1]), return_tensors='pt', padding="longest", truncation=True)
train_labels = get_labels(list(df_train.iloc[:,0]))

val_encodings = tokenizer(list(df_test.iloc[:,-1]), return_tensors='pt', padding="longest",truncation=True)
val_labels = get_labels(list(df_test.iloc[:,0]))



In [ ]:

class DataLoader(torch.utils.data.Dataset):
    def __init__(self, encodings,labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        input_ids = self.encodings['input_ids'][idx]
        attention_mask = self.encodings["attention_mask"][idx]
        label = self.labels[0][idx]
        return input_ids, attention_mask,label

    def __len__(self):
        return len(self.labels[0])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
train_dataset = DataLoader(train_encodings,train_labels)
val_dataset = DataLoader(val_encodings,val_labels)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= 16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size= 16, shuffle=True)

In [ ]:
from transformers import BertConfig, BertModel 
bert = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(768,156)
        self.fc2 = nn.Linear(156,3)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x,dim=1)
        return x


In [ ]:
model = Net()

In [ ]:
epochs = 3
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()



## Train the model and validate it after each epoch.
## Provide the train-val loss graph.
total_train_losses = []
total_train_acc= []

total_val_losses = []
total_val_acc = []

# *****START CODE
for epoch in range(1,epochs+1):
    ##TRAINING##
    model.train()
    train_losses = []
    train_acc = []



    for batch in train_dataloader:        
        input_ids_batch, attention_mask_batch, lbl_batch = batch
        
        optimizer.zero_grad()

        bert_rep = bert(input_ids_batch, attention_mask=attention_mask_batch)[0][:,0,:]
        outputs = model(bert_rep)
        loss = criterion(outputs,lbl_batch)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        
        pred = outputs.argmax(axis=1)
        train_acc.append(accuracy_score(lbl_batch,pred))
        

        
    train_loss_mean = np.mean(train_losses)
    total_train_losses.append(train_loss_mean)
    
    train_acc_mean = np.mean(train_acc)
    total_train_acc.append(train_acc_mean)
    # VALIDATION
    model.eval()
    val_losses = []
    validation_acc = []
    
    for val_batch in val_dataloader:
        input_ids_batch, attention_mask_batch, lbl_batch = val_batch
        bert_rep = bert(input_ids_batch, attention_mask=attention_mask_batch)[0][:,0,:]
        outputs = model(bert_rep)
        loss = criterion(outputs,lbl_batch)
        val_losses.append(loss.item())
        
        pred = outputs.argmax(axis=1)
        validation_acc.append(accuracy_score(lbl_batch,pred))
        
    
    val_loss_mean = np.mean(val_losses)
    total_val_losses.append(val_loss_mean)
    
    val_acc_mean = np.mean(validation_acc)
    total_val_acc.append(val_acc_mean)
    
    print('Epoch {}/{}): Train Loss: {:.6f} \t Validation Loss: {:.6f}'.format(
            epoch, epochs, train_loss_mean, val_loss_mean))
    print('Epoch {}/{}): Train Acc: {:.6f} \t Validation Acc: {:.6f}'.format(
            epoch, epochs, train_acc_mean, val_acc_mean))
    
    
    
# *****END CODE

In [ ]:
test = pd.read_csv("data/devdata.csv",sep = "\t",header=None)

deps_test = []
for j in range(len(test)):
    try:
        current_dep = []
        parses = dep_parser.parse(test.iloc[j,4].split())
        context = test.iloc[j,2].split()
    #parses_list = list(dep_parser.parse(df.iloc[5,4].split()))
        u= [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses][0]
    except:
        pass
    for word in context: 
        get_first_link(word,u,current_dep)
    direct_links = len(current_dep)
    for i in range(direct_links):
        context = current_dep[i]
        get_second_link(context,u,current_dep)
    deps_test.append(list(set(current_dep)))


In [ ]:
test.iloc[:,4] = test.iloc[:,4].str.replace(r'[^\w\s]+', ' ')
test["terms"] = deps_test
test["new_sent"] = test.apply(lambda x : filter_sentence(x["terms"],x[4]),axis=1)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
#encoding = tokenizer(list(df_test.iloc[:,-1]), return_tensors='pt', padding="longest", truncation=True)

test_encodings = tokenizer(list(test.iloc[:,-1]), return_tensors='pt', padding="longest", truncation=True)
test_labels = get_labels(list(test.iloc[:,0]))[0]

bert_rep = bert(test_encodings["input_ids"], test_encodings["attention_mask"])[0][:,0,:]
outputs = model(bert_rep)
test_pred = np.array(outputs.argmax(axis=1))
accuracy = accuracy_score(test_labels,test_pred)
print(accuracy)